In [ ]:
from flask import Flask, request, jsonify
from Melody_generator_user_defined_ip import generate_melody

app = Flask(__name__)

@app.route('/generate_melody', methods=['POST'])
def generate_melody_route():
    song = request.json['song']
    melody = generate_melody(song)
    return jsonify({'melody': melody})

if __name__ == '__main__':
    app.run(debug=True)

In [1]:
import json
import numpy as np
import keras as keras
import music21 as m21
import warnings
warnings.filterwarnings("ignore")

In [2]:
import import_ipynb
import pre_processing

importing Jupyter notebook from pre_processing.ipynb


In [3]:
import json
import numpy as np
import keras as keras
import music21 as m21

In [4]:
SEQUENCE_LENGTH = 64
MAPPING_PATH = r"C:\Users\aswin\Downloads\Capstone - SP\music.json"

In [7]:
class MelodyGenerator:
    """A class that wraps the LSTM model and offers utilities to generate melodies."""

    def __init__(self, model_path=r"C:\Users\aswin\Downloads\Capstone - SP\Weights\weights-improvement-200-0.1385-bigger.hdf5"):
        """Constructor that initialises TensorFlow model"""

        self.model_path = model_path
        self.model = keras.models.load_model(model_path)

        with open(MAPPING_PATH, "r") as fp:
            self._mappings = json.load(fp)

        self._start_symbols = ["/"] * SEQUENCE_LENGTH


    def generate_melody(self, seed, num_steps, max_sequence_length, temperature):
        """Generates a melody using the DL model and returns a midi file.

        :param seed (str): Melody seed with the notation used to encode the dataset
        :param num_steps (int): Number of steps to be generated
        :param max_sequence_len (int): Max number of steps in seed to be considered for generation
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.

        :return melody (list of str): List with symbols representing a melody
        """

        # create seed with start symbols
  
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            onehot_seed = keras.utils.to_categorical(seed, num_classes=len(self._mappings))
            # (1, max_sequence_length, num of symbols in the vocabulary)
            onehot_seed = onehot_seed[np.newaxis, ...]

            # make a prediction
            probabilities = self.model.predict(onehot_seed)[0]
            # [0.1, 0.2, 0.1, 0.6] -> 1
            output_int = self._sample_with_temperature(probabilities, temperature)

            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break

            # update melody
            melody.append(output_symbol)

        return melody


    def _sample_with_temperature(self, probabilites, temperature):
        """Samples an index from a probability array reapplying softmax using temperature

        :param predictions (nd.array): Array containing probabilities for each of the possible outputs.
        :param temperature (float): Float in interval [0, 1]. Numbers closer to 0 make the model more deterministic.
            A number closer to 1 makes the generation more unpredictable.

        :return index (int): Selected output symbol
        """
        predictions = np.log(probabilites) / temperature
        probabilites = np.exp(predictions) / np.sum(np.exp(predictions))

        choices = range(len(probabilites)) # [0, 1, 2, 3]
        index = np.random.choice(choices, p=probabilites)

        return index


    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):
        """Converts a melody into a MIDI file

        :param melody (list of str):
        :param min_duration (float): Duration of each time step in quarter length
        :param file_name (str): Name of midi file
        :return:
        """

        # create a music21 stream
        stream = m21.stream.Stream()

        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1

        # write the m21 stream to a midi file
        stream.write(format, file_name)



if __name__ == "__main__":
  
    song = m21.converter.parse(r'C:\Users\aswin\Downloads\Capstone - SP\erk\deut0705.krn')
    parts = song.getElementsByClass(m21.stream.Part)
    measures_part0 = parts[0].getElementsByClass(m21.stream.Measure)
    key = measures_part0[0][4]
    # estimate key using music21
    if not isinstance(key, m21.key.Key):
        key = song.analyze("key")

    # get interval for transposition. E.g., Bmaj -> Cmaj
    if key.mode == "major":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("C"))
    elif key.mode == "minor":
        interval = m21.interval.Interval(key.tonic, m21.pitch.Pitch("A"))
    
    song = song.transpose(interval)

    list_1 = []
    for event in song.flat.notesAndRests:
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi # 60
            list_1.append(symbol)
        elif isinstance(event, m21.note.Rest):
            symbol = "r"
            list_1.append(symbol)
    #print(list_1)

    time_step= 0.25
    encoded_song = []
    for event in song.flat.notesAndRests:
        # handle notes
        if isinstance(event, m21.note.Note):
            symbol = event.pitch.midi # 60
        # handle rests
        elif isinstance(event, m21.note.Rest):
            symbol = "r"
        # convert the note/rest into time series notation
        steps = int(event.duration.quarterLength / time_step)
        for step in range(steps):
            # if it's the first time we see a note/rest, let's encode it. Otherwise, it means we're carrying the same
            # symbol in a new time step
            if step == 0:
                encoded_song.append(symbol)
            else:
                encoded_song.append("_")

    encoded_song = " ".join(map(str, encoded_song))

    seed = encoded_song[:45]
    last_two = seed[-2:]
    if not last_two.isdigit():
        idx = seed.rfind('_')  # find the index of the last occurrence of '_'
        if idx != -1:
            # check if the next two characters form a continuous sequence of numbers
            if seed[idx+1].isdigit() and seed[idx+2].isdigit():
                seed = seed[:idx+3]
            else:
                seed = seed[:idx+1] + '  ' 
    print(seed)

    mg = MelodyGenerator()
    #seed = "67 _ 67 _ 67 _ _ 65 64 _ 64 _ 64 _ _"
    #seed2 = "67 _ _ _ _ _ 65 _ 64 _ 62 _ 60 _ _ _"
    melody = mg.generate_melody(seed, 200, SEQUENCE_LENGTH, 0.1)
    print(melody)
    mg.save_melody(melody)

64 _ 67 _ 64 _ 64 _ 64 _ 64 _ 62 _ 62 _ 62 _  
1/1 [==============================] - 0s 18ms/step
['64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '64', '_', '62', '_', '62', '_', '62', '_', '60', '_', '64', '_', '67', '_', '69', '_', '69', '_', '67', '_', '67', '_', '64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '64', '_', '62', '_', '62', '_', '62', '_', '60', '_', '64', '_', '67', '_', '69', '_', '69', '_', '67', '_', '67', '_', '64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '64', '_', '62', '_', '62', '_', '65', '_', '64', '_', '60', '_', '62', '_', '64', '_', '60', '_', '_', '_', 'r', '_', '64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '64', '_', '62', '_', '62', '_', '62', '_', '60', '_', '64', '_', '67', '_', '69', '_', '69', '_', '67', '_', '67', '_', '64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '67', '_', '64', '_', '64', '_', '64', '_', '64', '_', '62', '_', '62', '_', '62', '_', '60', '_', 